In [4]:
# Importing Necessary Libraries

import pandas as pd
import numpy as np
import keras
import tensorflow
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense

In [5]:
data = pd.read_csv('data/train.txt', sep=';')
data.columns = ['Text', 'Emotions']
data.head(3)

,Text,Emotions
0,i can go from feeling so hopeless to so damned...,sadness
1,im grabbing a minute to post i feel greedy wrong,anger
2,i am ever feeling nostalgic about the fireplac...,love


In [6]:
texts = data['Text'].tolist()
labels = data['Emotions'].tolist()

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)

In [8]:
sequences = tokenizer.texts_to_sequences(texts)
max_length = max([len(seq) for seq in sequences])
padded_sequence = pad_sequences(sequences, maxlen=max_length)
padded_sequence

array([[   0,    0,    0, ...,    3,   21, 1254],
       [   0,    0,    0, ...,    2,  494,  437],
       [   0,    0,    0, ...,   29,    5, 3496],
       ...,
       [   0,    0,    0, ...,    3,  101, 1331],
       [   0,    0,    0, ...,  339,    8,   42],
       [   0,    0,    0, ...,   25, 3585,   12]])

In [9]:
# Encode the String Labels to Integers
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

In [10]:
# One Hot Encode the labels
one_hot_labels = keras.utils.to_categorical(labels)
one_hot_labels

array([[0., 0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       ...,
       [0., 0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0.]])

In [11]:
# Splitting dataset into training and testing
xtrain, xtest, ytrain, ytest = train_test_split(padded_sequence, one_hot_labels, test_size=0.2)

In [14]:
# Defining the Model
from sklearn import metrics


model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1,
                    output_dim=128, input_length=max_length))
model.add(Flatten())
model.add(Dense(units=128, activation='relu'))
model.add(Dense(units=len(one_hot_labels[0]), activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(xtrain, ytrain, epochs=10, batch_size=32, validation_data=(xtest, ytest))

g:\Portfolio Projects\Data Science\Day 1-Text Emotion Classification\env\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 45s 97ms/step - accuracy: 0.3586 - loss: 1.5603 - val_accuracy: 0.7247 - val_loss: 0.8287
Epoch 2/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 39s 97ms/step - accuracy: 0.8627 - loss: 0.4508 - val_accuracy: 0.8144 - val_loss: 0.5673
Epoch 3/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 37s 93ms/step - accuracy: 0.9815 - loss: 0.0722 - val_accuracy: 0.8125 - val_loss: 0.6205
Epoch 4/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 35s 86ms/step - accuracy: 0.9952 - loss: 0.0247 - val_accuracy: 0.8191 - val_loss: 0.6448
Epoch 5/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 34s 86ms/step - accuracy: 0.9972 - loss: 0.0148 - val_accuracy: 0.8181 - val_loss: 0.6879
Epoch 6/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 35s 88ms/step - accuracy: 0.9971 - loss: 0.0117 - val_accuracy: 0.8134 - val_loss: 0.7122
Epoch 7/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 37s 91ms/step - accuracy: 0.9972 - loss: 0.0162 - val_accuracy: 0.8119 - val_loss: 0.7922
Epoch 8/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 35s 86ms/step - accuracy: 0.9975 - loss: 0.0114 - 

In [24]:
# Test text
input_text = "Today has been one of the most amazing days of my life. I got a promotion at work, and my colleagues threw a surprise celebration for me. I feel so happy and appreciated!"

In [25]:
# Preprocessing input text
input_sequence = tokenizer.texts_to_sequences([input_text])
padded_input = pad_sequences(input_sequence, maxlen=max_length)
prediction = model.predict(padded_input)
predicted_label = label_encoder.inverse_transform([np.argmax(prediction[0])])
predicted_label[0]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step


'joy'